In [40]:
import h5py
import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.transform import Rotation as R
import manifpy

In [41]:
h5f = h5py.File("data/30_minutes.h5", "r")

In [46]:
id = 1
raw_data = np.array(h5f[f"tracker{id}"]) # time, xyz q_wxyz
q_ws = raw_data[:, 3:4]
raw_data[:, 3:6] = raw_data[:, 4:7]
raw_data[:, 6:7] = q_ws #xyz q_xyzw
max_sp = np.array([5.0, 5.0, 5.0, np.pi, np.pi, np.pi])/980.0
max_sp = np.array([np.inf]*6)

process_data = []
raw_velocity = []
filtered_SE3 = None
alpha = 1.0
for t in range(raw_data.shape[0]):
    if t==0:
        filtered_SE3 = manifpy.SE3(raw_data[t])
    else:
        se3 = manifpy.SE3(raw_data[t]) - filtered_SE3
        se3_clipped = se3.coeffs_copy()

        se3_clipped = np.clip(se3_clipped, -max_sp, max_sp)
        se3_clipped = manifpy.SE3Tangent(se3_clipped)

        filtered_SE3 = filtered_SE3 + (se3_clipped*alpha)
        raw_velocity.append(se3_clipped.coeffs_copy()*alpha)
    
    process_data.append(np.hstack([filtered_SE3.translation(), filtered_SE3.quat()]))

process_data = np.vstack(process_data)
raw_velocity = np.vstack(raw_velocity)*980.0

In [47]:
%matplotlib qt 

start_time_step = 0
max_time_step = raw_velocity.shape[0]

time = np.arange(max_time_step) / 1000
index = np.arange(max_time_step) + start_time_step
plt.rcParams["figure.figsize"] = (20, 8)
plt.subplot(2,3,1)
plt.plot(time, raw_velocity[index, 0], 'b')
# plt.plot(time, process_data[index, 0], '--r')

plt.subplot(2,3,2)
plt.plot(time, raw_velocity[index, 1], 'b')
# plt.plot(time, process_data[index, 1], '--r')

plt.subplot(2,3,3)
plt.plot(time, raw_velocity[index, 2], 'b')
# plt.plot(time, process_data[index, 2], '--r')

plt.subplot(2,3,4)
plt.plot(time, raw_velocity[index, 3], 'b')
# plt.plot(time, rpy_mat_proccessed[:,0], '--r')

plt.subplot(2,3,5)
plt.plot(time, raw_velocity[index, 4], 'b')
# plt.plot(time, rpy_mat_proccessed[:,1], '--r')

plt.subplot(2,3,6)
plt.plot(time, raw_velocity[index, 5], 'b')
# plt.plot(time, rpy_mat_proccessed[:,2], '--r')

plt.show()
# plt.plot(data[:,0])

In [1]:
import pandas as pd

In [6]:
pd.read_csv("../data/test.csv")

,x0,x1,x2,x3,x4
0,0,0,0,0,0
0,0,0,0,0,0
0,0,0,0,0,0
0,0,0,0,0,0
0,0,0,0,0,0
0,0,0,0,0,0
0,0,0,0,0,0
0,0,0,0,0,0
0,0,0,0,0,0
0,0,0,0,0,0
